<h1> Sequential Graph </h1>

### Intro
Often we want to break down a complex task into smaller steps. Node A does something, passes it to Node B, which does something else.

In this lesson, we'll build a simple pipeline: **First Node** -> **Second Node**.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph

### The State
We need `name` and `age` as inputs, and `result` as output.

In [ ]:
class AgentState(TypedDict):
    name: str
    age: str
    result: str

### Nodes
- **First Node**: Takes the name and starts the message string.
- **Second Node**: Takes the existing message and appends the age info.

In [ ]:
def firstNode(state: AgentState) -> AgentState:
    """
    Step 1: Adds the name to the result
    """ 
    state["result"] = f"Hey {state["name"]}, "
    return state

In [ ]:
def secondNode(state: AgentState) -> AgentState:
    """ 
    Step 2: Appends the age to the result.
    """ 
    # Notice we are using state['result'] which was set by the previous node
    state["result"] = state["result"] + f"Your age is: {state["age"]} !"
    return state

### Connecting the Dots
Here is where the magic happens. We add both nodes, but instead of finishing after the first one, we add an **edge** from 'first' to 'second'.

This tells LangGraph: "When 'first' is done, go to 'second'."

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("first", firstNode)
graph.add_node("second", secondNode)

graph.set_entry_point("first")
graph.add_edge("first", "second") # The Sequential Link
graph.set_finish_point("second")

app = graph.compile()

### Step-by-Step Visualization
See how the arrow goes from first to second? That's our pipeline.

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

### Run the Pipeline
We provide `name` and `age`, and the graph processes them in order.

In [ ]:
result = app.invoke({"name": "Debu", "age": 22})
print(result["result"])